In [ ]:
import numpy as np 
import pandas as pd 
import glob

In [ ]:
# global variables
country_names = ['Brazil', 'Bulgaria', 'Chile', 'China', 'Colombia', 
               'Croatia', 'Korea', 'Malaysia', 'Mexico', 'Philippines', 
               'Poland', 'Russia', 'SAfrica', 'Thailand', 'Turkey']

In [ ]:
"""
    compare log likelihood, own script, author table, author fortran code
"""

# read own estimation
own_df = pd.read_csv("logs/log_table4.csv", names=['ALP', 'BET', 'SIG', 'ALPH', 'BETH', 'SIG1', 'SIG2', 'own_LLK', 'FVALL'])
own_df['country'] = country_names
own_df = own_df.set_index('country')

# read table on estimation result
paper_llk = [998.06, 1290.36, 1481.87, 1567.52, 978.67, 
             1401.06, 1461.71, 1450.38, 1304.45, 1023.43, 
             1600.79, 1161.84, 1332.30, 1429.16, 1028.71]
# add to main dataframe
df = (-1*own_df['own_LLK']).to_frame().assign(table4_LLK=paper_llk)

# read fortran result
temp = []
for country in country_names:
    temp.append(pd.read_csv(f"fortran_results/OUTC2_{country}.TXT", 
                        delim_whitespace=True, 
                        header=None, 
                        names=['ALP', 'BET', 'SIG', 'ALPH', 'BETH', 'SIG1', 'SIG2', 'fortran_LLK', 'FVALL']))
fortran_df = pd.concat(temp)
fortran_df['country'] = country_names
fortran_df = fortran_df.set_index('country')
fortran_df['fortran_LLK'] = -1 * fortran_df['fortran_LLK']
df['fortran__LLK'] = fortran_df['fortran_LLK']

# save comparison in df to local 
df[['own_LLK', 'fortran__LLK', 'table4_LLK']].to_csv('outputs/comparison/LLK.csv')

In [ ]:
"""
    compare own and fortran intermediate result (interpolated model CDS prices)
"""

mean_diff = []
compare_cols = ['XLAM', 'CDS1X', 'CDS1XP', 'CDS3X', 'CDS3XP', 'CDS5X', 'CDS5XP']
for country in country_names:
    fortran_cds = pd.read_csv(f"fortran_results/OUTC1_{country}.TXT", 
                                delim_whitespace=True, 
                                header=None, 
                                names=['XLAM','CDS1Y','CDS1X','CDS1XP','CDS3','CDS3X','CDS3XP','CDS5','CDS5X','CDS5XP'])
    own_cds = pd.read_csv(f"outputs/{country}_CDS.csv")
    # compute average abs difference in each cell
    # exclude columns that copy CDS market data
    mean_diff.append((np.abs(fortran_cds[compare_cols] - own_cds[compare_cols]).values.sum())/own_cds[compare_cols].size)

In [ ]:
pd.DataFrame({'CDS_mean_abs_diff':mean_diff, 'country':country_names}).set_index('country').to_csv('outputs/comparison/model_CDS_values.csv')

In [1]:
"""
    numba experimentation
"""
import importlib
import numpy as np
import main

# importlib.reload(main)

XGUESSES = [[3.52, -0.83, 0.75, -3.38, 0.69, 0.0059, 0.0035]]

In [2]:
# 47.6s
# -998.2932754704742
# main.FUNC(PARM=XGUESSES[0], ICAR_1=0, logging=False, log_file_path="", print_flag=True, test_flag=False)

In [4]:
# 37.3
# 13.7
# 9.2
main.FUNC(PARM=XGUESSES[0], ICAR_1=0, logging=False, log_file_path="", print_flag=True, test_flag=False)

[ 3.52e+00 -8.30e-01  7.50e-01 -3.38e+00  6.90e-01  5.90e-03  3.50e-03],-998.2932754704742, 0.00398082755737068


-998.2932754704742